In [1]:
import pandas as pd
from neuralforecast.core import NeuralForecast
from neuralforecast.models import TimesNet
import torch
import matplotlib.pyplot as plt

/home/maybe/miniconda3/envs/task2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-07-16 14:43:42,699	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-07-16 14:43:42,774	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
# 2. ОПРЕДЕЛЕНИЕ ПАРАМЕТРОВ — должны совпадать с обучающей конфигурацией!
horizon = 40
input_size = 200

In [3]:
# 1. Подготовка (формат ваших новых или тестовых данных)
df = pd.read_csv('evs1_012_10_bar_harmonics_median_new.csv', parse_dates=['date'])
df = df.rename(columns={'date': 'ds', 'value': 'y'})
df['unique_id'] = 'id_1'

In [4]:
# 3. Восстановление структуры модели
model = TimesNet(
    h=horizon,
    input_size=input_size,
    max_steps=60,
    learning_rate=1e-3,
    scaler_type='standard'
)
nf = NeuralForecast(models=[model], freq='D')

Seed set to 1


In [5]:
# 4. Загрузка весов модели
model.model.load_state_dict(torch.load('timesnet_state_dict.pth'))
# 5. Если весь контейнер сохраняли через nf.save/nf.load:
# nf = NeuralForecast.load('neuralforecast_nf.pkl')

/tmp/ipykernel_6409/1442950293.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.model.load_state_dict(torch.load('timesnet_state_dict.pth'))


<All keys matched successfully>

In [6]:
# 6. Прогнозирование
forecast_df = nf.predict(df=df)

Exception: You must fit the model before predicting.

In [ ]:
# 7. Визуализация (при необходимости)
actual = df.set_index('ds')['y']
forecast = forecast_df.set_index('ds')['TimesNet']

plt.figure(figsize=(12, 6))
plt.scatter(actual.index, actual.values, color='black', label='Исторические данные', s=10)
plt.scatter(forecast.index, forecast.values, color='blue', label='TimesNet прогноз', s=10)
plt.xlabel('Дата')
plt.ylabel('Значение')
plt.title('Исторические данные и прогноз TimesNet')
plt.legend()
plt.grid(True)
plt.show()
